In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import logging
from pathlib import Path
import textwrap
import copy
from PIL import Image, ImageDraw, ImageFont
# from waveshare_epd import epd2in7b
from waveshare_epd import epd5in83

In [4]:
import epdlib.constants as constants
import epdlib.layouts as layouts

In [5]:
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [6]:
class ImageBlock:
    '''Class for creating 1 bit image of word-wrapped text.
    
    ImageBlock objects contain an image block bounded by size `area` that can 
    be used to assemble a larger composite image. ImageBlocks are aware of 
    their position (`abs_coordinates`) within the larger image and the position 
    of the formatted (resized) image. The position of the formatted image, 
    relative to the larger image is calculated and stored in `img_coordinates`. 
        
    
    Attributes:
        image (PIL.Image): Image to be formatted and resized to fit within
            area
        area (:obj:`tuple` of :obj: `int`): x, y integer dimensions of 
            maximum area in pixles
        abs_coordinates (:obj:`tuple` of `int`): x, y integer coordinates of image area
            within a larger image
        img_coordinates (:obj:`tuple` of `int`): x, y calculated integer 
            coordinates of image within the larger image - calculation is based 
            on centering, image size
        hcenter (boolean): True - horizontal-align image within the area, 
            False - left-align image
        vcenter (boolean): True - vertical-align image within the area,
            False - top-align image
        padding (int): amount of padding between resized image and edge of area
    '''
    def __init__(self, image=None, area=None, abs_coordinates=(0, 0), hcenter=False, vcenter=True, padding=0):
        '''Constructor for ImageBlock Class.
        
        Args:
            image (PIL.Image): Image to be formatted and resized to fit within
                area
            area (:obj:`tuple` of :obj: `int`): x, y integer dimensions of 
                maximum area in pixles
            abs_coordinates (:obj:`tuple` of `int`): x, y integer coordinates of image area
                within a larger image
            hcenter (boolean, optional): True - horizontal-align image within the area, 
                False - left-align image
            vcenter (boolean, optional): True - vertical-align image within the area,
                False - top-align image
            padding (int, optional): amount of padding between resized image and edge of area'''
        self.area = area
        self.abs_coordinates = abs_coordinates
        self.hcenter = hcenter
        self.vcenter = vcenter
        self.padding = padding
        self.image = image

    def update(self, update=None):
        '''Generic update function the object with the incoming data.
        
        Method is used by other class objects to update all image blocks 
        
        Args:
            update (str): path to image file
                
        Returns:
            bool: True upon success'''
        if update:
            try:
                self.image = update
            except Excepiton as e:
                logging.error(f'failed to update: {e}')
                return False
            return True
    
    @property
    def area(self):
        ''':obj:`tuple` of :obj:`int`: maximum area of imageblock'''
        return self._area

    @area.setter
    def area(self, area):
        if self._coordcheck(area):
            self._area = area
            logging.debug(f'maximum area: {area}')
        else:
            raise ValueError(f'bad area value: {area}')    

    @property
    def abs_coordinates(self):
        ''':obj:`tuple` of :obj:`int`: absolute_coordinates of area within larger image.
        
        Setting `abs_coordinates` atomatically sets `img_coordinates` to the same value.
        '''
        return self._abs_coordinates
    
    @abs_coordinates.setter
    def abs_coordinates(self, abs_coordinates):
        if self._coordcheck(abs_coordinates):
            self.img_coordinates = abs_coordinates
            self._abs_coordinates = abs_coordinates
        else:
            raise ValueError(f'bad absolute coordinates: {abs_coordinates}')
    
    @property
    def image(self):
        ''':obj:`PIL.Image`: resizes, and centers image to fit within the `area`
        
        Setting/Updating `image` recalculates `img_coordinates`'''
        return self._image
    
    @image.setter
    def image(self, image):
        if not image:
            self._image = None
            return None
        logging.debug(f'formatting image: {image}')
        dim = min(self.area)-self.padding
        logging.debug(f'set image dimensions: {dim}')
        size = (dim, dim)
        im = Image.open(image)
        im.convert(mode='L', colors=2)
        im.thumbnail(size)
        self.dimensions = im.size
        x_new, y_new = self.abs_coordinates
        
        if self.hcenter:
            x_new = self.abs_coordinates[0] + round(self.area[0]/2 - self.dimensions[0]/2)
        if self.vcenter:
            y_new = self.abs_coordinates[1] + round(self.area[1]/2 - self.dimensions[1]/2)
        
        if self.hcenter or self.vcenter:
            self.img_coordinates = (x_new, y_new)
        logging.debug(f'set img_coordinates: {self.img_coordinates}')
            
        self._image = im
        return im
    
    def _coordcheck(self, coordinates):
        '''Check that coordinates are of type int and positive.
        
        Args:
            coordinates (:obj:`tuple` of :obj: `int`)
            
        Raises:
            TypeError: `coordinates` are not a list or tuple
            TypeError: `coordinates` elements are not an integer
            ValueError: `coordinates` are not >=0
            '''
        if not isinstance(coordinates, (tuple, list)):
            raise TypeError(f'must be type(list, tuple): {coordinates}')
        for i, c in enumerate(coordinates):
            if not isinstance(c, int):
                raise TypeError(f'must be type(int): {c}')
                return False
            if c < 0:
                raise ValueError(f'coordinates must be positive: {c}')
                return False
        return True            

In [7]:
class TextBlock:
    '''Class for creating 1 bit image of word-wrapped text.
    
    TextBlock objects contain an image of formatted text bounded by size `area` 
    that can be used to assemble a larger composite image. The text is 
    word-wrapped and formatted based on area size, font size, and centering rules.
    TextBlocks are aware of their position (`abs_coordinates`) within a larger 
    image and the position of the formatted text. The position of the text, 
    relative to the larger image is calculated and stored in `img_coordinates`. 
    
    
    Attributes:
        area (:obj:`tuple` of :obj: `int`): x, y integer dimensions of 
            maximum area in pixles
        text (str): text to be formatted and converted into image
        font (str): path to TTF font
        font_size (int): size of font in points
        max_lines (int): maximum number of lines to return after wrapping text
        maxchar (int): maximum number of characters per line (calculated if not provided
        abs_coordinates (:obj:`tuple` of `int`): x, y integer coordinates of image area
            within a larger image
        hcenter (boolean): True - horizontal-align text within the area, 
            False - left-align text
        vcenter (boolean): True - vertical-align text within the area,
            False - top-align text
        img_coordinates (:obj:`tuple` of `int`): x, y calculated integer 
            coordinates of image within the larger image - calculation is based 
            on centering, image size
        chardist (:obj:dict of :obj:str : :obj:float): dictionary describing
            letter frequency in a language for calcualting maxchar
        image (PIL.Image): Image to be formatted and resized to fit within
            area
    '''
    
    def __init__(self, area=(600, 448), text=' ', font=None, font_size=24, max_lines=1,
                maxchar=None, hcenter=False, vcenter=False, abs_coordinates=(0,0), 
                chardist=None):
        '''Constructor for TextBlock Class.
        
        Args:
            area (:obj:`tuple` of :obj: `int`): x, y integer dimensions of 
                maximum area in pixles
        text (str): text to be formatted and converted into image
        font (str): path to TTF font
        font_size (int): size of font in points
        max_lines (int): maximum number of lines to return after wrapping text
        maxchar (int): maximum number of characters per line (calculated if not provided
        hcenter (boolean, optional): True - horizontal-align text within the area, 
            False - left-align text
        vcenter (boolean, optional): True - vertical-align text within the area,
            False - top-align text
        abs_coordinates (:obj:`tuple` of `int`): x, y integer coordinates of image area
            within a larger image            
        chardist (:obj:dict of :obj:str : :obj:float): dictionary describing
            letter frequency in a language for calcualting maxchar
            '''    
        
        self.area = area
        if font:
            self.font = ImageFont.truetype(str(Path(font).resolve()), size=font_size)
        else:
            self.font = ImageFont.truetype(str(Path(constants.FONT).resolve()), size=font_size)
        
        if chardist:
            self._chardist = chardist
        else:
            self._chardist = constants.USA_CHARDIST
        self.max_lines = max_lines
        self.maxchar = maxchar
        
        self.hcenter = hcenter
        self.vcenter = vcenter
        
        self.abs_coordinates = abs_coordinates
        
        self.image = None
        
        self.text = text
    
    def update(self, update=None):
        '''Generic update function the object with the incoming data.
        
        Method is used by other class objects to update all image blocks 
        
        Args:
            update (str): path to image file
                
        Returns:
            bool: True upon success'''
        if update:
            try:
                self.text = update
            except Excepiton as e:
                logging.error(f'failed to update: {e}')
                return False
            return True

    @property
    def area(self):
        ''':obj:`tuple` of :obj:`int`: maximum area of imageblock'''        
        return self._area

    @area.setter
    def area(self, area):
        if self._coordcheck(area):
            self._area = area
            logging.debug(f'maximum area: {area}')
        else:
            raise ValueError(f'bad area value: {area}')
     
    @property
    def maxchar(self):
        '''int: maximum number of characters per line - if no value 
                is provided, this will be calculated
        
        If no value is provided, a random string of characters is generated based on the
        frequency tables: `chardist`. The default distribution is American English. 
        Based on this string the maximum number of characters for a given font and font size.
            
        '''
        return self._maxchar
    
    @maxchar.setter
    def maxchar(self, maxchar):
        if maxchar:
            self._maxchar = maxchar
        else:
            s = ''
            n = 1000
            # create a string of characters containing the letter distribution
            for char in self._chardist:
                s = s+(char*int(self._chardist[char]*n))
            s_length = self.font.getsize(s)[0] # string length in Pixles
            avg_length = s_length/len(s)
            maxchar = round(self.area[0]/avg_length)
            self._maxchar = maxchar
            logging.debug(f'maximum characters per line: {maxchar}')
    
    @property
    def abs_coordinates(self):
        ''':obj:`tuple` of :obj:`int`: absolute_coordinates of area within larger image.
        
        Setting `abs_coordinates` atomatically sets `img_coordinates` to the same value.'''
        return self._abs_coordinates
    
    @abs_coordinates.setter
    def abs_coordinates(self, abs_coordinates):
        if self._coordcheck(abs_coordinates):
            self._abs_coordinates = abs_coordinates
            self.img_coolrdinates = abs_coordinates
            logging.debug(f'absolute coordinates: {abs_coordinates}')
        else:
            raise ValueError(f'bad absoluote coordinates: {abs_coordinates}')
            
    
    @property
    def text(self):
        ''':obj:`str`: raw text to be formatted.
        
        setting or resetting this property will also set the following attributes
            text (str): unformatted raw text
            text_formatted (list): of (str): wrapped text
            image (:obj:`PIL.Image`): image based on wrapped and formatted text
            img_coordinates (:obj:`tuple` of :obj:`int`): coordinates of text image adjusted 
                for size of textblock'''
        return self._text
    
    @text.setter
    def text(self, text):
        self._text = text
        self.text_formatted = self.text_formatter()
        self.image = self._text2image()
    
    def text_formatter(self, text=None, max_lines=None, maxchar=None):
        '''format text using word-wrap strategies. 
        
        Formatting is based on number of lines, area size and maximum characters per line
        
        Args:
            text (str): raw text
            maxchar (int): maximum number of characters on each line
            max_lines (int): maximum number of lines
            
        Returns:
            :obj:`list` of :obj:`str`
        '''
        if not text:
            text = self.text
        if not maxchar:
            maxchar = self.maxchar
        if not max_lines:
            max_lines = self.max_lines
        
        wrapper = textwrap.TextWrapper(width=maxchar, max_lines=max_lines)
        formatted = wrapper.wrap(text)
        logging.debug(f'formatted list:\n {formatted}')
        return(formatted)
    
    def _text2image(self):
        '''produces 1 bit image containing wrapped text.
        
        calling this method will set `img_coordinates`
            based on image size, hcenter and vcenter rules
        
        Sets:
            dimensions (tuple): of (int) - dimensions of text image
            img_coordinates (tuple): of int - absolute coordinates of text image 
        
        Returns:
            :obj:`PIL.Image`
        '''
        
        # determine the extents of the text block image
        y_total = 0
        x_max = 0
        
        for line in self.text_formatted:
            x, y = self.font.getsize(line)
            y_total += y # accumulate the total height
            if x > x_max:
                x_max = x # find the longest line
        
        # set dimensions of the text block image
        self.dimensions = (x_max, y_total)
        logging.debug(f'text image dimensions: {self.dimensions}')
        
        # build image
        image = Image.new('1', self.dimensions, 255)
        # get a drawing context
        draw = ImageDraw.Draw(image)
        
        y_total = 0
        for line in self.text_formatted:
            x_pos = 0
            x, y = self.font.getsize(line)
            if self.hcenter:
                logging.debug(f'h-center line: {line}')
                x_pos = round(self.dimensions[0]/2-x/2)
            draw.text((x_pos, y_total), line, font=self.font)
            y_total += y
        
        # set image coordinates
        new_x, new_y = self.abs_coordinates
        if self.hcenter:
            logging.debug(f'h-center image coordinates')
            new_x = self.abs_coordinates[0] + round(self.area[0]/2 - self.dimensions[0]/2)

            
        if self.vcenter:
            logging.debug(f'v-center image coordinates')
            new_y = self.abs_coordinates[1] + round(self.area[1]/2 - self.dimensions[1]/2)
        
        if self.hcenter or self.vcenter:
            logging.debug(f'image coordinates {(new_x, new_y)}')
            self.img_coordinates = (new_x, new_y)
        
        return image
    
    
    def _coordcheck(self, coordinates):
        '''Check that coordinates are of type int and positive.
        
        Args:
            coordinates (:obj:`tuple` of :obj: `int`)
            
        Raises:
            TypeError: `coordinates` are not a list or tuple
            TypeError: `coordinates` elements are not an integer
            ValueError: `coordinates` are not >=0
        '''
        if not isinstance(coordinates, (tuple, list)):
            raise TypeError(f'must be type(list, tuple): {coordinates}')
        for i, c in enumerate(coordinates):
            if not isinstance(c, int):
                raise TypeError(f'must be type(int): {c}')
                return False
            if c < 0:
                raise ValueError(f'coordinates must be positive: {c}')
                return False
        return True

In [8]:
class Screen:
    '''Class for interfacting with WaveShare EPD screens.
    
    `Screen` creates an object that provides methods for assembling images
    and updating a WaveShare EPD.
    
    Attributes:
        resolution (:obj:`tuple` of :obj: `int`): resolution of EPD
        elements (:obj:`list` of :obj:`ImageBlock` or `TextBlock`): images to be assembled
        image (:obj:`PIL.Image`): composite image to be written to screen
        epd (:obj:`waveshare.epd`): waveshare EPD object 
    '''
    def __init__(self, resolution=(600, 448), elements=[], epd=None):
        '''Constructor for Screen class.
        
        Args:
            resolution (:obj:`tuple` of :obj: `int`): resolution of EPD
            elements (:obj:`list` of :obj:`ImageBlock` or `TextBlock`): images to be assembled
            image (:obj:`PIL.Image`): composite image to be written to screen
            epd (:obj:`waveshare.epd`): waveshare EPD object'''
        self.resolution = resolution
        self.elements = elements
        self.image = self.clearScreen()
        self.epd = epd
        
    def clearScreen(self):
        '''Sets a clean base image for building screen layout.
        
        Returns:
            :obj:PIL.Image
        '''
        image = Image.new('L', self.resolution, 255)
        return image
    
    def concat(self, elements=None):
        '''Concatenate multiple image objects into a single composite image
        
        Args:
            elements (:obj:`list` of :obj:`ImageBlock` or `TextBlock`) - if none are
                provided, use the existing elements
                
        Sets:
            image (:obj:`PIL.Image`): composite of all members of `elements`
            
        Returns:
            image (:obj:`PIL.Image`)
        '''
        self.image = self.clearScreen()
        if elements:
            elements = elements
        else:
            elements = self.elements
            
        for e in elements:
            logging.debug(f'pasing image at: {e.img_coordinates}')
            self.image.paste(e.image,  e.img_coordinates)
        return(self.image)
    
    def initEPD(self):
        '''Initialize the connection with the EPD Hat.
        
        Returns:
            bool: True if successful
        '''
        if not self.epd:
            raise UnboundLocalError('no epd object has been assigned')
        try:
            self.epd.init()
        except Exception as e:
            logging.error(f'failed to init epd: {e}')
        return True
    
    def clearEPD(self):
        '''Clear the EPD screen.
        
        Raises:
            UnboundLocalError: no EPD has been intialized
        
        Returns:
            bool: True if successful'''
        if not self.epd:
            raise UnboundLocalError('no epd object has been assigned')
        try:
            self.epd.Clear();
        except Exception as e:
            logging.error(f'failed to clear epd: {e}')
        return True
    
    def writeEPD(self, image=None, sleep=True):
        '''Write an image to the EPD.
        
        Args:
            image (:obj:`PIL.Image`, optional): if none is provided use object `image`
            sleep (bool): default - True; put the EPD to low power mode when done writing
            
        Returns:
            bool: True if successful
        '''
        epd = self.epd
        if not self.epd:
            raise UnboundLocalError('no epd object has been assigned')
        try:
            epd.display(epd.getbuffer(self.image))
            if sleep:
                epd.sleep()
        except Exception as e:
            logging.error(f'failed to write to epd: {e}')
            return False
        return True
        

In [89]:
class Layout:
    def __init__(self, resolution=(600, 448), layout=None, font=constants.FONT):
        self.resolution = resolution
        self.font = str(Path(font).resolve())
        self.layout = copy.deepcopy(layout)
        self.images = None

    def _check_keys(self, dictionary={}, values={}):
        logging.debug('checking key/values')
        for k, v in values.items():
            try:
                dictionary[k]
            except KeyError as e:
                logging.debug(f'missing key: {k}; adding and setting to {v}')
                dictionary[k] = v
        return dictionary
    
    def _scalefont(self, font=None, lines=1, text="W", dimensions=(100, 100)):
        
        if font:
            font = str(Path(font).resolve())
        else:
            font = str(Path(self.font).resolve())
            
        logging.debug('calculating font size')
        logging.debug(f'using font at path: {font}')
        
        
        # start calculating at size 1
        fontsize = 1
        y_fraction = .7
        target = dimensions[1]/lines*y_fraction
        testfont = ImageFont.truetype(font, fontsize)
        fontdim = testfont.getsize(text)
        
        logging.debug(f'target Y fontsize: {target}')
        
        # work up until font covers img_fraction of the resolution return one smaller than this as the fontsize
        while fontdim[1] < target:
            fontdim = testfont.getsize(text)
            if fontdim[1] > dimensions[1]:
                logging.warn('font Y dimension is larger than Y area; bailing out')
                break
            fontsize += 1
            testfont = ImageFont.truetype(font, fontsize)
            
        # back off one 
        fontsize -= 1
        logging.debug(f'fontsize: {fontsize}')
        return fontsize
    
    @property
    def layout(self):
        return self._layout
    
    @layout.setter
    def layout(self, layout):
        logging.debug(f'calculating values from layout for resolution {self.resolution}')
        if not layout:
            logging.info('no layout provided')
            self._layout = None
        else:
            self._layout = self.calculate_layout(layout)
            self.set_images()
    
    
    def calculate_layout(self, layout):
        if not layout:
            return None
        l = layout
        resolution = self.resolution
        # required values that will be used in calculating the layout
        values = {'image': None, 'max_lines': 1, 'padding': 0, 'width': 1, 'height': 1, 
                  'abs_coordinates': (None, None), 'hcenter': False, 'vcenter': False, 'relative': False, 
                  'font': self.font, 'fontsize': None, 'dimensions': None}        
        for section in l:
            logging.debug(f'***{section}***')
            this_section = self._check_keys(l[section], values)
                    
            dimensions = (round(resolution[0]*this_section['width']), 
                          round(resolution[1]*this_section['height']))
            
            this_section['dimensions'] = dimensions
            logging.debug(f'dimensions: {dimensions}')       
        
            # set the thumbnail_size to resize the image
            if this_section['image']:
                maxsize = min(this_section['dimensions'])-this_section['padding']*2
                this_section['thumbnail_size'] = (maxsize, maxsize)
            
            # calculate the relative position if needed
            # if either of the coordinates are set as "None" - attempt to calculate the position
            if this_section['abs_coordinates'][0] is None or this_section['abs_coordinates'][1] is None:
                logging.debug(f'has calculated position')
                # store coordinates
                pos = []
                # check each value in relative section
                for idx, r in enumerate(this_section['relative']):
                    if r == section:
                        # use the value from this_section
                        pos.append(this_section['abs_coordinates'][idx])
                    else:
                        # use the value from another section
                        pos.append(l[r]['dimensions'][idx] + l[r]['abs_coordinates'][idx])
                
                # save the values as a tuple
                this_section['abs_coordinates']=(pos[0], pos[1])
            else:
                logging.debug('has explict position')
                ac= this_section['abs_coordinates']
            logging.debug(f'abs_coordinates: {ac}')
                          
            # calculate fontsize
            if this_section['max_lines']:
                if not this_section['font']:
                    this_section['font'] = self.font
                          
                if not this_section['fontsize']:
                    this_section['fontsize'] = self._scalefont(font=this_section['font'], 
                                                               dimensions=this_section['dimensions'],
                                                               lines=this_section['max_lines'])    

            l[section] = this_section    
        return l
                              
    def set_images(self):
        '''create dictonary of all image blocks with using the current set layout
        
            Sets
            ----
                ::blocks :dict of: TextBlock(), ImageBlock()
            '''
                          
        
        layout = self.layout
        
        blocks = {}
        for sec in layout:
            logging.debug(f'***{sec}***)')
            section = layout[sec]
            # any section with max lines accepts text
            if section['max_lines']:
                logging.debug('set text block')
                blocks[sec] = TextBlock(area=section['dimensions'], text='.', font=section['font'], 
                                       font_size=section['fontsize'], max_lines=section['max_lines'],
                                       hcenter=section['hcenter'], vcenter=section['vcenter'],
                                       abs_coordinates=section['abs_coordinates'])
            if section['image']:
                logging.debug('set image block')
                blocks[sec] = ImageBlock(image=None, abs_coordinates=section['abs_coordinates'], 
                                         area=section['dimensions'], hcenter=section['hcenter'],
                                         vcenter=section['vcenter'], padding=section['padding'])
        self.blocks = blocks
                              
    def update_contents(self, updates=None):
        if not updates:
            logging.debug('nothing to do')
        
        for key, val in updates.items():
            self.blocks[key].update(val)
                

In [87]:
constants.FONT

'../fonts/Open_Sans/OpenSans-Regular.ttf'

In [90]:
# l = Layout(layout=layouts.twoColumn)
l = Layout(layout=layouts.threeRow, font='./fonts/Open_Sans/OpenSans-Regular.ttf')

DEBUG:root:calculating values from layout for resolution (600, 448)
DEBUG:root:***title***
DEBUG:root:checking key/values
DEBUG:root:missing key: dimensions; adding and setting to None
DEBUG:root:dimensions: (600, 179)
DEBUG:root:has explict position
DEBUG:root:abs_coordinates: (0, 0)
DEBUG:root:calculating font size
DEBUG:root:using font at path: /home/pi/src/slimpi_epd/fonts/Open_Sans/OpenSans-Regular.ttf
DEBUG:root:target Y fontsize: 62.65
DEBUG:root:fontsize: 59
DEBUG:root:***coverart***
DEBUG:root:checking key/values
DEBUG:root:missing key: dimensions; adding and setting to None
DEBUG:root:dimensions: (360, 269)
DEBUG:root:has calculated position
DEBUG:root:abs_coordinates: (0, 0)
DEBUG:root:***artist***
DEBUG:root:checking key/values
DEBUG:root:missing key: dimensions; adding and setting to None
DEBUG:root:dimensions: (240, 90)
DEBUG:root:has calculated position
DEBUG:root:abs_coordinates: (0, 0)
DEBUG:root:calculating font size
DEBUG:root:using font at path: /home/pi/src/slimpi_

In [91]:
uA = {'title': "Steam Engine",
      'artist': "My Morning Jacket",
      'album': "It Still Moves",
      'status': "Playing",
      'coverart': "./cover.jpg"}
uB = {'title': "This Tornado Loves You",
      'artist': "Neko Case",
      'album': "Middle Cyclone",
      'status': "Paused",
      'coverart': "./ozo_cover.jpg"}


In [108]:
l.update_contents(uB)

DEBUG:root:formatted list:
 ['This Tornado Loves', 'You']
DEBUG:root:text image dimensions: (534, 130)
DEBUG:root:h-center line: This Tornado Loves
DEBUG:root:h-center line: You
DEBUG:root:h-center image coordinates
DEBUG:root:v-center image coordinates
DEBUG:root:image coordinates (33, 24)
DEBUG:root:formatted list:
 ['Neko Case']
DEBUG:root:text image dimensions: (149, 33)
DEBUG:root:v-center image coordinates
DEBUG:root:image coordinates (360, 207)
DEBUG:root:formatted list:
 ['Middle Cyclone']
DEBUG:root:text image dimensions: (213, 40)
DEBUG:root:v-center image coordinates
DEBUG:root:image coordinates (360, 294)
DEBUG:root:formatted list:
 ['Paused']
DEBUG:root:text image dimensions: (202, 65)
DEBUG:root:v-center image coordinates
DEBUG:root:image coordinates (360, 371)
DEBUG:root:formatting image: ./ozo_cover.jpg
DEBUG:root:set image dimensions: 259
DEBUG:root:set img_coordinates: (50, 186)


In [100]:
l.blocks

{'title': <__main__.TextBlock at 0xb34f1790>,
 'coverart': <__main__.ImageBlock at 0xaf8b44b0>,
 'artist': <__main__.TextBlock at 0xaf8b4890>,
 'album': <__main__.TextBlock at 0xaf8b4e50>,
 'status': <__main__.TextBlock at 0xaf8b4770>}

In [106]:
e = epd5in83.EPD()
s = Screen()
s.epd = e
s.initEPD()

DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release


True

In [97]:
s.initEPD()
s.clearEPD()

DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release
DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release


True

In [110]:
# s.elements = [l.blocks['title'], l.blocks['artist'], l.blocks['album'], l.blocks['status'], l.blocks['coverart']]
s.initEPD()
s.elements = l.blocks.values()
s.concat()
s.writeEPD()

DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release
DEBUG:root:pasing image at: (33, 24)
DEBUG:root:pasing image at: (50, 186)
DEBUG:root:pasing image at: (360, 207)
DEBUG:root:pasing image at: (360, 294)
DEBUG:root:pasing image at: (360, 371)
DEBUG:root:imwidth = 600  imheight =  448 
DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release
DEBUG:root:e-Paper busy
DEBUG:root:e-Paper busy release
DEBUG:root:spi end
DEBUG:root:close 5V, Module enters 0 power consumption ...


True